In [1]:
import swarmrl as srl

from swarmrl.observables.top_down_image import TopDownImage
from jax import numpy as np
import matplotlib.pyplot as plt
from swarmrl.components import Colloid
import open3d as o3d
import logging
import flax.linen as nn
from swarmrl.tasks.dummy_task import DummyTask
import optax
from swarmrl.actions.mpi_action import MPIAction
from swarmrl.engine.gaurav_sim import *
from swarmrl.trainers.global_continuous_trainer import GlobalContinuousTrainer as Trainer
import pint


2024-09-13 12:09:03.714594: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.4 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
logging.basicConfig(level=logging.WARNING)


resolution=1280
number_of_gaussians=2
action_dimension=8
rafts = o3d.io.read_triangle_mesh("coloured_rafts.ply")

obs = TopDownImage(
    np.array([10000.0, 10000.0, 0.1]), image_resolution=np.array([resolution]*2), particle_type=0, custom_mesh=rafts, is_2D=True, save_images=False
)
task = DummyTask()


class ActoCriticNet(nn.Module):
    """A simple dense model."""

    @nn.compact
    def __call__(self, x):
        x = nn.Conv(features=32, kernel_size=(3, 3), strides=(3, 3))(x)
        x = nn.relu(x)
        x = nn.Conv(features=64, kernel_size=(3, 3), strides=(3, 3))(x)
        x = nn.relu(x)
        x = nn.max_pool(x, window_shape=(10, 10), strides=(10, 10))
        x = x.flatten()
        y = nn.Dense(features=2)(x)
        x = nn.Dense(features=2)(x)
        x = nn.relu(x)
        y = nn.relu(y)

        y = nn.Dense(features=12)(y)
        x = nn.Dense(features=12)(x)
        x = nn.relu(x)
        y = nn.relu(y)
        y = nn.Dense(features=1)(x)  # Critic
        x = nn.Dense(features=number_of_gaussians*action_dimension*2)(x)  # Actor
        #pass output designed for variance through relu function (last number_of_gaussians*action_dimension)
        # x = x.at[number_of_gaussians * action_dimension:].set(nn.relu(x.at[number_of_gaussians * action_dimension:].get()))
        x = nn.relu(x)
        return x, y


exploration_policy = srl.exploration_policies.RandomExploration(probability=0.0) # check this

# Define a sampling_strategy
sampling_strategy = srl.sampling_strategies.ContinuousGaussianDistribution()

# Value function to use
value_function = srl.value_functions.GlobalExpectedReturns(gamma=0.1, standardize=True)

# Define the model
actor_critic = ActoCriticNet()



[Open3D INFO] EGL headless mode enabled.
FEngine (64 bits) created at 0x830b7800010 (threading is enabled)
EGL(1.5)
OpenGL(4.1)


In [3]:
ureg = pint.UnitRegistry()
Q_ = ureg.Quantity

# Define parameters in SI units
params = GauravSimParams(
            ureg=ureg,
            box_length=Q_(10000, "micrometer"),
            time_step=Q_(1e-3, "second"),
            time_slice=Q_(1, "second"),
            snapshot_interval=Q_(0.002, "second"),
            raft_radius=Q_(150, "micrometer"),
            raft_repulsion_strength=Q_(1e-7, "newton"),
            dynamic_viscosity=Q_(1e-3, "Pa * s"),
            fluid_density=Q_(1000, "kg / m**3"),
            lubrication_threshold=Q_(15, "micrometer"),
            magnetic_constant=Q_(4 * np.pi * 1e-7, "newton /ampere**2"),
            capillary_force_data_path=pathlib.Path(
                "/work/clohrmann/mpi_collab/capillaryForceAndTorque_sym6"
            ),  # TODO
        )

# Initialize the simulation system
system_runner = GauravSim(params=params, out_folder="./", with_precalc_capillary=True,save_h5=True)
mag_mom = Q_(1e-8, "ampere * meter**2")
for i in range(20):
    system_runner.add_colloids(pos = [np.random.rand()*10000,np.random.rand()*10000, 0]* ureg.micrometer, alpha = np.random.rand()*2*np.pi, magnetic_moment = 1E-8* ureg.ampere * ureg.meter**2)



In [4]:

network = srl.networks.ContinuousFlaxModel(
    flax_model=actor_critic,
    optimizer=optax.adam(learning_rate=0.01),
    input_shape=(1,resolution,resolution,1), #1 are required for CNN
    sampling_strategy=sampling_strategy,
    exploration_policy=exploration_policy,
    number_of_gaussians=number_of_gaussians,
    action_dimension=action_dimension,
)
loss = srl.losses.GlobalPolicyGradientLoss(value_function=value_function)

protocol = srl.agents.MPIActorCriticAgent(
    particle_type=0,
    network=network,
    task=task,
    observable=obs,
    loss=loss,
)
rl_trainer = Trainer([protocol])


                             ActoCriticNet Summary                              
┏━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ path    ┃ module        ┃ inputs          ┃ outputs         ┃ params         ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│         │ ActoCriticNet │ float32[1,1280… │ - float32[32]   │                │
│         │               │                 │ - float32[1]    │                │
├─────────┼───────────────┼─────────────────┼─────────────────┼────────────────┤
│ Conv_0  │ Conv          │ float32[1,1280… │ float32[1,427,… │ bias:          │
│         │               │                 │                 │ float32[32]    │
│         │               │                 │                 │ kernel:        │
│         │               │                 │                 │ float32[3,3,1… │
│         │               │                 │                 │                │
│         │               │

In [5]:
rl_trainer.perform_rl_training(system_runner, 20, 10)

Output()

self.current_action=MPIAction(amplitudes=array([0.0009516, 0.01     ], dtype=float32), frequencies=array([0., 0.], 
dtype=float32), phases=array([22.797235  ,  0.07230361], dtype=float32), offsets=array([0.02154703, 0.00950223], 
dtype=float32))

self.current_action=MPIAction(amplitudes=array([0.00206274, 0.01      ], dtype=float32), 
frequencies=array([4.7349296, 1.6970372], dtype=float32), phases=array([25.479315 ,  2.6517944], dtype=float32), 
offsets=array([0.02041507, 0.01131005], dtype=float32))

self.current_action=MPIAction(amplitudes=array([1.3163175e-03, 1.7266397e-07], dtype=float32), 
frequencies=array([1.4264587e-03, 1.9456568e+01], dtype=float32), phases=array([19.128511, 28.050154], 
dtype=float32), offsets=array([0.01428498, 0.02212538], dtype=float32))

self.current_action=MPIAction(amplitudes=array([0.00398466, 0.01      ], dtype=float32), 
frequencies=array([1.8039424, 4.8124647], dtype=float32), phases=array([21.396143,  0.      ], dtype=float32), 
offsets=array([0.02239195, 0.00891743], dtype=float32))

...

self.current_action=MPIAction(amplitudes=array([7.7154407e-05, 1.3054408e-05], dtype=float32), 
frequencies=array([0.        , 0.00019596], dtype=float32), phases=array([0.        , 0.01982396], dtype=float32), 
offsets=array([0., 0.], dtype=float32))

self.current_action=MPIAction(amplitudes=array([3.2302389e-05, 1.1733893e-05], dtype=float32), 
frequencies=array([0.01808855, 0.00147777], dtype=float32), phases=array([0.       , 0.0349041], dtype=float32), 
offsets=array([8.5189757e-05, 6.1792603e-06], dtype=float32))

self.current_action=MPIAction(amplitudes=array([1.1883867e-04, 1.2505409e-05], dtype=float32), 
frequencies=array([0.0203278, 0.       ], dtype=float32), phases=array([0., 0.], dtype=float32), offsets=array([0.,
0.], dtype=float32))

self.current_action=MPIAction(amplitudes=array([8.1602883e-07, 7.0606534e-06], dtype=float32), 
frequencies=array([0.00502931, 0.00486002], dtype=float32), phases=array([0., 0.], dtype=float32), 
offsets=array([0.0000000e+00, 4.7260373e-06], dtype=float32))

self.current_action=MPIAction(amplitudes=array([5.886445e-06, 5.714120e-06], dtype=float32), 
frequencies=array([0.0048357 , 0.00422619], dtype=float32), phases=array([0.04808242, 0.        ], dtype=float32), 
offsets=array([0.0000000e+00, 2.8312818e-06], dtype=float32))